In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125095")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125095
Azure region: southcentralus
Subscription id: de47103e-2da6-4f5e-88fc-d18b27fd249b
Resource group: aml-quickstarts-125095


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.compute.amlcompute(class)?view=azure-ml-py#provisioning-configuration-vm-size-----vm-priority--dedicated---min-nodes-0--max-nodes-none--idle-seconds-before-scaledown-none--admin-username-none--admin-user-password-none--admin-user-ssh-key-none--vnet-resourcegroup-name-none--vnet-name-none--subnet-name-none--tags-none--description-none--remote-login-port-public-access--notspecified--
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps =  RandomParameterSampling({
    '--C': uniform(0.5, 1.5),
    '--max_iter': choice(50, 150)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.085)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='.',
compute_target=ws.compute_targets['cpu-cluster'],
entry_script='train.py'
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
hyperparameter_sampling=ps,
policy=policy,
primary_metric_name='Accuracy',
primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
max_total_runs=10,
max_concurrent_runs=2)
print('HyperDriveConfig has been created!')

HyperDriveConfig has been created!


In [11]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

my_run = exp.submit(config=hyperdrive_config)
RunDetails(my_run).show()
my_run.wait_for_completion(show_output=True)
assert(my_run.get_status()=="Completed")

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_56589c3a-e9ad-4cac-bdf6-f5639a00e924
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_56589c3a-e9ad-4cac-bdf6-f5639a00e924?wsid=/subscriptions/de47103e-2da6-4f5e-88fc-d18b27fd249b/resourcegroups/aml-quickstarts-125095/workspaces/quick-starts-ws-125095

Streaming azureml-logs/hyperdrive.txt

"<START>[2020-11-07T23:16:24.642087][API][INFO]Experiment created<END>\n""<START>[2020-11-07T23:16:25.132393][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space<END>\n""<START>[2020-11-07T23:16:25.339997][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_56589c3a-e9ad-4cac-bdf6-f5639a00e924
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_56589c3a-e9ad-4cac-bdf6-f5639a00e924?wsid=/subscriptions/de47103e-2da6-4f5e-88fc-d18b27fd249b/resourcegroups/aml-quickstarts-125095/workspaces/quick-starts-ws-125095



In [12]:
import joblib
# Get your best run and save the model from that run.

best_run = my_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
run_definition = parameter_values.get("runDefinition")

print('Best Run Id: ', best_run.id)
print('Accuracy:', best_run_metrics['Accuracy'])
print('Arguments for best run:', run_definition.get("arguments"))
print()
print(best_run.get_file_names())
print()
# Register model
model = best_run.register_model(model_name="my_best_run", model_path="outputs/model.joblib")

# Check model
from azureml.core import Model
for model in Model.list(ws):
    print("Model Name: {}\n".format(model.name))
    print(model)


Best Run Id:  HD_56589c3a-e9ad-4cac-bdf6-f5639a00e924_0
Accuracy: 0.9123573682932751
Arguments for best run: ['--C', '1.426879702270979', '--max_iter', '150']

['azureml-logs/55_azureml-execution-tvmps_e27f26dbed2b5d015d9890450001a72379f04b535bd53adb01e44c0c89adc017_d.txt', 'azureml-logs/65_job_prep-tvmps_e27f26dbed2b5d015d9890450001a72379f04b535bd53adb01e44c0c89adc017_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_e27f26dbed2b5d015d9890450001a72379f04b535bd53adb01e44c0c89adc017_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/110_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']

Model Name: my_best_run

Model(workspace=Workspace.create(name='quick-starts-ws-125095', subscription_id='de47103e-2da6-4f5e-88fc-d18b27fd249b', resource_group='aml-quickstarts-125095'), name=my_best_run, id=my_best_run:2, version=2, tags={}, properties={})
Model Name: my_best

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

web_path = ['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv']
ds = TabularDatasetFactory.from_delimited_files(path=web_path)

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x,y)

train_data = pd.concat([x_train, y_train], axis=1)
valid_data = pd.concat([x_test, y_test], axis=1)

In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=train_data,
    validation_data=valid_data,
    label_column_name='y', 
    n_cross_validations=5)

In [16]:
# Submit your automl run

exp = Experiment(workspace=ws, name="udacity-project-AutoML")

my_automl_run = exp.submit(config=automl_config)
RunDetails(my_automl_run).show()
my_automl_run.wait_for_completion(show_output=True)
assert(my_automl_run.get_status()=="Completed")

ValidationException: ValidationException:
	Message: Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. 
Required version/Installed version
cryptography<=3.1.1/cryptography 3.2
zipp<=3.3.1/zipp 3.4.0
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Install the required versions of packages using the requirements file. Requirements file location: /anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/automl/core/validated_linux_requirements.txt. Alternatively, use remote target to avoid dependency management. \nRequired version/Installed version\ncryptography<=3.1.1/cryptography 3.2\nzipp<=3.3.1/zipp 3.4.0",
        "inner_error": {
            "code": "NotSupported",
            "inner_error": {
         

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###